# Image Generator

In [ ]:
#| default_exp image_generator

In [ ]:
#| export
from Cloud2DImageConverter import spherical_projection as sp
from Cloud2DImageConverter import data
from matplotlib import pyplot as plt
from tqdm import tqdm
from PIL import Image
import numpy as np
import shutil
import time
import os

In [ ]:
#| export
FOLDER_PATH = "../point_clouds/semantic_kitti/"
VELODYNE_PATH = FOLDER_PATH+"velodyne"
LABEL_PATH = FOLDER_PATH+"labels"
RESULTS_FOLDER =  "../results/"

BATCH_SIZE = 500
MAX_LEN = len(os.listdir(VELODYNE_PATH))

In [ ]:
#| export
if os.path.exists(RESULTS_FOLDER):
    shutil.rmtree(RESULTS_FOLDER)
    
os.makedirs(RESULTS_FOLDER)
os.makedirs(RESULTS_FOLDER+"/projections")
os.makedirs(RESULTS_FOLDER+"/labels")

In [ ]:
#| export
def define_range(batch, batch_size, max_len):
    start = batch - batch_size
    end = batch
    if (max_len - batch) < batch_size:
        end = max_len
    return start, end

In [ ]:
#| export
def do_projection(point_cloud):
    projection_dict = {"reflectance": [],"label": [], "depth": []}
    for points in point_cloud:
        unlabaled, labaled, depth = sp.spherical_projection(points)
        labaled = sp.colored_matrix_with_label(labaled)
        projection_dict["reflectance"].append(unlabaled)
        projection_dict["label"].append(labaled)
        projection_dict["depth"].append(depth)
    return projection_dict

In [ ]:
#| export
def save_image(matrix, save_path):
    img = Image.fromarray(matrix)
    final_path = os.path.join(RESULTS_FOLDER+save_path)
    file_number = len(os.listdir(final_path)) + 1
    file_name = f"{final_path}/{file_number:06d}.png"
    img.save(file_name)

def create_images(projection_dict):
    # Itera sobre cada posição do dicionário agrupando junto as matrizes de mesmo indice
    for zip_dict in zip(projection_dict["reflectance"], projection_dict["label"], projection_dict["depth"]):
        # Alterna entre as chaves e os indices respectivos a cada chave
        aux_dict = {}
        for index, key in enumerate(projection_dict.keys()):
            matrix = zip_dict[index]
            if key != "label":
                norm_matrix = ((matrix - matrix.min()) / (matrix.max() - matrix.min())) * 255
                matrix = norm_matrix.astype(np.uint8)
            aux_dict[key] = matrix
        black_matrix = np.zeros((64, 1024), dtype=np.uint8)
        aux_dict["final_matrix"] = np.dstack((np.array(aux_dict["reflectance"]), np.array(aux_dict["depth"]), black_matrix))
        save_image(aux_dict["final_matrix"], "/projections")
        save_image(aux_dict["label"], "/labels")

In [ ]:
#| export
for batch in tqdm(range(BATCH_SIZE, MAX_LEN, BATCH_SIZE), desc="Batch:"):
    # Define o intervalo de carregamento dos dados
    start, end = define_range(batch, BATCH_SIZE, MAX_LEN)
    # Define as listas contendo os nomes dos arquivos
    velodyne_list = sorted(os.listdir(VELODYNE_PATH))[start:end]
    label_list = sorted(os.listdir(LABEL_PATH))[start:end]
    # Carrega os dados 
    point_cloud = data.load_data(VELODYNE_PATH, LABEL_PATH, velodyne_list, label_list)
    # Realiza a projeção esférica
    projection_dict = do_projection(point_cloud)
    # Cria e salva as imagens
    create_images(projection_dict)

Batch:: 100%|█████████████████████████████████████████████████████████████████| 5/5 [00:25<00:00,  5.08s/it]


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()